In [104]:
def date_modifier(time_stamp):
    dt_obj = datetime.datetime.fromtimestamp(time_stamp)
    return datetime.datetime.strftime(dt_obj,"%d%B_%I:%M_%p")


def send_email(df):
    from email.mime.text import MIMEText
    from email.mime.application import MIMEApplication
    from email.mime.multipart import MIMEMultipart
    from smtplib import SMTP
    import smtplib
    import sys


    recipients = ['maneeshgowdkosuru@gmail.com'] 
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = "Daily Weather Report"
    msg['From'] = 'ksaimaneesh123@gmail.com'


    html = """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(df.to_html())

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login("ksaimaneesh123@gmail.com", "whiumfxpsjhomsyi")
    server.sendmail(msg['From'], emaillist , msg.as_string())
    server.close()


def send_daily_weather_report_email(location='Manubolu'):
    location = location.replace(' ','%20')
    
    import requests
    import datetime
    import pandas as pd
    
    API_key = 'ef8c2c992a449699483af5da15c3a0c7'
    req = requests.get(f'https://api.openweathermap.org/data/2.5/forecast?q={location}&appid={API_key}&units=metric')
    
    li = []
    try:
        for item in req.json()['list']:
            date = date_modifier(item['dt'])
            temp = str(item['main']['temp'])
            weather = item['weather'][0]['description']
            humidity = item['main']['humidity']
            cloudy_perc = item['clouds']['all']
            wind_speed = str(item['wind']['speed'])
            li.append([date,weather,temp,humidity,cloudy_perc,wind_speed])

        df = pd.DataFrame(columns=['Date and Time','Weather ','Temperature(C)','Humidity(%)','Cloudy(%)','Wind(m/s)'],data=li)

        send_email(df)
    except:
        return 'Cant find such location'

In [105]:
send_daily_weather_report_email('New york')